In [1]:
import pandas as pd

In [2]:
finance = pd.read_csv('../data/step1/finance.csv')
returns = pd.read_csv('../data/step1/returns.csv')
for df in [finance, returns]:
    df['Time'] = pd.to_datetime(df['Time'])
stk = pd.DataFrame(returns['Stkcd'].unique(), columns=['Stkcd'])

In [3]:
# Group: Size_BM, Size_OP, Size_INV (2[A, B] * 3[a, b, c])
stk_group = []
T = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
for t in T:
    df = pd.read_csv('../data/step2/{}.csv'.format(t))
    for label1 in ['A', 'B']:
        for item in ['BM', 'OP', 'INV']:
            for label2 in ['a', 'b', 'c']:
                stk = df[(df['Size'] == label1) & (df[item] == label2)]['Stkcd'].tolist()
                stk_group.append((str(t) + '_' + item + '_' + label1 + label2, stk))
stk_group = dict(stk_group)

In [4]:
# Calculate Weighted Return Rate R
# R = sum(msmvosd * mretwd) / sum(msmvosd)
# This part may take a bit long time.

R = []
for x in stk_group:
    [year, item, label] = x.split('_')
    year = int(year)
    m = 7
    
    df = pd.DataFrame(index=None)
    for st in stk_group[x]:
        tmp = returns[returns['Stkcd'] == st]
        df = pd.concat([df, tmp])
    
    for y in [year - 2, year - 1, year]:
        while (y < year and m <= 12) or (y == year and m <= 7):
            r = df[(df['Time'].dt.year == y) & (df['Time'].dt.month == m)][['Stkcd', 'Msmvosd', 'Mretwd']]
            r['S*R'] = r['Msmvosd'] * r['Mretwd']
            ans = r['S*R'].sum() / r['Msmvosd'].sum()
            # print(str(y) + '_' + str(y) + str(m) + '_' + item + '_' + label, ans)
            R.append((str(year) + '_' + str(y) + str(m) + '_' + item + '_' + label, ans))
            m += 1
        m = 1

In [5]:
R = pd.DataFrame(R, index=None)
R.columns = ['label', 'R']
R.to_csv('../data/step3/Weighted_Return_Rate.csv')